In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
import cPickle
from sklearn.utils import  shuffle

In [ ]:
#获取训练与测试数据
def get_data():
    with open('./transfer_feature/train.pkl') as f:
        train = cPickle.load(f)
    with open('./transfer_feature/test.pkl') as f:
        test = cPickle.load(f)
    return train,test

In [ ]:
#构建分类器，此处是一个4层DNN
def model(X):
    fc1 = slim.fully_connected(X,512,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='fc1',weights_initializer=initializer)
    fc2 = slim.fully_connected(fc1,1024,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='fc2',weights_initializer=initializer)
    fc3 = slim.fully_connected(fc2,512,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='fc3',weights_initializer=initializer)
    fc4 = slim.fully_connected(fc3,4,normalizer_fn=slim.batch_norm,\
        activation_fn=None,scope='fc4',weights_initializer=initializer)
    return fc4

In [ ]:
train,test = get_data()

X = tf.placeholder("float", [None, 512])
Y = tf.placeholder("int32", [None])

initializer = tf.contrib.layers.xavier_initializer()

trX = train['data']
trY = train['label']
teX = test['data']
teY = test['label']

logits = model(X)
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits=logits)

cost = tf.reduce_mean(entropy)
train_op = tf.train.AdamOptimizer(0.0002).minimize(cost)
precision= tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits,Y,1),tf.float32))

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch=32
for i in range(100):
    trX,trY = shuffle(trX,trY)
    for start, end in zip(range(0, len(trX), batch), range(batch, len(trX)+1, batch)):
        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
    print(i,sess.run(precision, feed_dict={X: teX,Y:teY}))